# LAB 4
# Webscraping


In this lab you are supposed to write a code that webscrapes data from the website <a href="http://books.toscrape.com">http://books.toscrape.com</a>. This website was made for the purpose of practicing webscraping.<br>
In fact, the code is already prepared for you and you just need to replace the commented text: <br> 
<font color=#339988><i><b>
\#YOUR CODE HERE!!!</b></i></font> <br>
with appropriate codes.

**When you are done, submit both your modified Lab4.ipynb file and its html version Lab4.html. To convert ipynb to html, inside Jupyter dashboard (where your ipynb file is open), go to File $\to$ Donwload as $\to$ HTML (.html)**

In [9]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# vector of column names to be used in a data frame
columnHeaders = ['Title', 'Price', 'Rating', 'Link']

#Create an empty pandas dataframe called "df" with column headers given in the vector columnHeaders
##################
df = pd.DataFrame(columns=columnHeaders)
##################

The following code cell contains a while-loop for grabbing different pages.
First page to begin with is <a href="http://books.toscrape.com/catalogue/page-1.html">http://books.toscrape.com/catalogue/page-1.html</a> <br>
You will need to create a string variable `url` which in the first iteration will be exactly <br>
http://books.toscrape.com/catalogue/page-1.html <br>
In the second iteration of the while-loop, the `url` variable will take the value <br>
http://books.toscrape.com/catalogue/page-2.html, <br>
in the third, <br>
http://books.toscrape.com/catalogue/page-3.html <br>
and so on.

In [ ]:
# Create a counter for the while loop to follow, will denote page number in loop
counter = 1

# Create empty list to store each book 
bookData = []

# Create a loop to loop through all the different pages on the webpage
while counter <= 50:
    
    # Define a dynamic url to get to the next page in every iteration of the while-loop.
    # The variable url should be a string, which changes from iteration to iteration
    # in the way described in the markdown cell above
    #########################
    url = "http://books.toscrape.com/catalogue/page-" + str(counter) + ".html"
    #########################
    
    # Use the requests library to send a get-request for the current url
    page = requests.get(url)
    
    # Use Beautiful Soup library to create a soup object that allows us to parse the html content of the page we got
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Find all the tags that contain the information we would like to collect per book using Beautiful Soup find_all method
    # and store them into lists
    price = soup.find_all('p', attrs={'class':'price_color'}) # p-tags (paragraphs) whose class is named 'price_color'
    title = soup.find_all('h3')  # h3-tags (level 3 headers) 
    rating = soup.find_all('p', attrs={'class':'star-rating'})  # p-tags whose class is named 'star-rating'
    #create a list called 'link' which consists of div-tags (div is one of the html tags), whose class 
    #is named 'image_container'; this list should be created in similar fashion as lists price, title and rating
    ##################
    link =  soup.find_all('div', attrs={'class':'image_container'})
    ##################
    
    # Note: The length of the lists created in the last step are all the same, since those specific tags only exist per book.
    # iterating through the ranges of the list, append each book's information to the dataframe created earlier
   
    ## Iterating through each book on current page and storing them into the empty list bookData.
    for i in range(len(price)):
        
        books = {
            
            'Title': title[i].a.get('title'),
            
            'Price': price[i].get_text(),
            
            'Rating': rating[i].get('class')[1],
            
            'Link': link[i].a.get('href')
        }
        
        bookData.append(books)
    
    #Print Counter to ensure loop is running properly, optional
    print(counter, url)
    
    #Increase counter at the end of the loop to change the url during next loop iteration
    counter = counter + 1
    
    # filling our empty DataFrame with our books. 
    df = pd.DataFrame(bookData)

### From now on it's about pandas 

In [ ]:
# Print the head of the dataframe to sanity check
print(df.head())

In [ ]:
# Print the tail of the dataframe to sanity check (there shold be 1000 observations in total, so the last index is 999)
print(df.tail())

You can see that Price column consists of prices in pounds. The first symbol in each component of this column is £ and the values are of str type (i.e. the entries are strings). Check this by printing the type of 1st entry of the Price column (i.e. run the next cell).

In [ ]:
type(df['Price'][0])

We now want to convert the price into US dollars, assuming that one pound is 1.3 dollars (£1 = $1.3).

In [ ]:
#drop the first character (i.e. symbol £) from all entries of column Price
df['Price'] = df['Price'].str[1:]

#convert the type of Price entries from strings (str) to reals (float)
df['Price'] = df['Price'].astype(float)

df.head(3)

In [ ]:
#convert prices from pounds to dollars (change values of Price column by factor of 1.3)
##################
df['Price'] = df['Price'] * 1.3
##################

In [ ]:
#checking whether conversion was correct (compare it with the output of the previous cell)
df.head(3)

In [ ]:
#change column name 'Price' into '$Price'
df.rename(columns={'Price':'$Price'}, inplace = True)

### In the following set of cells you need to find the title and the price of the least expensive book and the most expensive book

In [ ]:
# Create a data frame df_sorted which consists of data from df, sorted by price 
# from the cheapest to the most expensive (i.e. in ascending order) 
##################

df_sorted = df.sort_values(by='$Price', ascending=True)
##################


#print first 10 rows of the data frame df_sorted (note the index labels, which are from the original data frame)
##################
df_sorted.head(10)
##################

# Reset the index so that df_sorted is re-indexed 0,1,2,3,... with the new order of the list
df_sorted = df_sorted.reset_index(drop=True)
df_sorted.head(10) #print first 10 rows of df_sorted

In [ ]:
# Print the title of the cheapest book
print(df_sorted['Title'][0])

# Print the price of the cheapest book
##################
print(df_sorted['$Price'][0])
##################

In [ ]:
# Print the title of the most expensive book
##################
print(df_sorted['Title'][999])
##################

# Print the price of the most expensive book
##################
print(df_sorted['$Price'][999])
##################

### In this part you need to plot the histogram of prices, as well as the bar plot of frequencies of books with various star-rating categories

In [ ]:
#Use matplotlib to plot a histogram of prices (in dollars) with 20 bins
##################
# Also create borders for the bins
plt.figure(figsize=(10,6))
plt.title('Price distribution')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.hist(df_sorted['$Price'], bins=20, edgecolor='black')
plt.show()
##################

In [ ]:
# finding the rating frequency distribution (count of books) for all star ratings.
# Rating Distribution found by getting the length of the "list" where the rating value equals the specified rating
print("One Star: " + str(len(df[df['Rating'] == 'One'])))
print("Two Star: " + str(len(df[df['Rating'] == 'Two'])))
print("Three Star: " + str(len(df[df['Rating'] == 'Three'])))
print("Four Star: " + str(len(df[df['Rating'] == 'Four'])))
print("Five Star: " + str(len(df[df['Rating'] == 'Five'])))

In [ ]:
# Create a list for the x and y axis labels for a bar plot
x_bar = ['One', 'Two', 'Three', 'Four', 'Five']
y_bar = [len(df[df['Rating'] == 'One']), len(df[df['Rating'] == 'Two']), len(df[df['Rating'] == 'Three']),
        len(df[df['Rating'] == 'Four']), len(df[df['Rating'] == 'Five'])]


# Use matplotlib to plot the bar plot
##################
plt.bar(x_bar, y_bar)
##################

# Change the x axis title, y axis title, and main title of plot
plt.xlabel("Star Rating")
plt.ylabel("Number of Books")
plt.title("Number of Books per Rating")

plt.show()  #needed in script files .py (eg in spyder)

### That's it! You are done!

### REMINDER: as mentioned at the beginning of this file, when you are done, submit both your modified Lab4.ipynb file and its html version Lab4.html. To convert ipynb to html, inside Jupyter dashbord (where your ipynb is open), go to File  →  Donwload as  →  HTML (.html)

### IMPORTANT: Save to html AFTER you run all the cells and the output is printed out.

<br>

### Appendix

##### Here is some more code, for getting the number of books in each category (get url links from main page for all catgories, go through this links and get the number of books per category)

In [ ]:
# Create a new data frame for catgories with specific column headers
columnHeaders = ['Category','Count']
cat_df = pd.DataFrame(columns = columnHeaders)

# Define main url to get all categories, get the page requested, and turn html content into a soup object
url = 'http://books.toscrape.com/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

# Find the section tag that contains all the category urls
categories = soup.find('ul', attrs={'class':'nav nav-list'})

# Find all the tags, within the categories variable, that contains the links
categories_list = categories.find_all('a')

# Get all the links from this list of tags
for i in range(len(categories_list)):
    categories_list[i] = categories_list[i].get('href')

# Iterate through the list of category htmls
for i in categories_list:
    # For each category, go to the page associted with that category and turn it into a soup object
    url = 'http://books.toscrape.com/' + i
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    # Find the category name and number of books within that category
    cat_title = soup.find('div', attrs={'class':'page-header action'}).h1.get_text()
    num_books = soup.find('form', attrs={'class':'form-horizontal'}).strong.get_text()
    
    # Append to the dataframe the category and number of books
    #cat_df = cat_df.append({'Category': cat_title, 'Count': num_books}, ignore_index = True)
    
    #We need to use pd.concat() instead of cat_df.append() to update the dataframe with the category and number of books
    cat_df = pd.concat([cat_df, pd.DataFrame({'Category': [cat_title], 'Count': [num_books]})], ignore_index  = True)
                       
                       

What is the distribution of books within each category? Here is the answer.

In [ ]:
# Sort the category list by count of books and print out the distribution
cat_df['Count'] = cat_df['Count'].astype(int)
cat_df = cat_df.sort_values(by=['Count'])
cat_df = cat_df.reset_index(drop=True)
print(cat_df)

### REMINDER: as mentioned at the beginning of this file, when you are done, submit both your modified Lab4.ipynb file and its html version Lab4.html. To convert ipynb to html, inside Jupyter dashbord (where your ipynb is open), go to File  →  Donwload as  →  HTML (.html)

### IMPORTANT: Save to html AFTER you run all the cells and the output is printed out.